<a href="https://colab.research.google.com/github/martinpius/Computer-Vission/blob/main/MLP_from_scratch_in_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>> You are in CoLaB with torch version {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)} {e}\n>>>> please correct {type(e)} and reload")
def time_fmt(t: float = 123.719)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h} min: {m:>02} sec: {s:>05.2f}"
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
print(f">>>> time formating\t..................\n>>>> time elapsed\t{time_fmt()}")

Mounted at /content/drive
>>>> You are in CoLaB with torch version 1.8.1+cu101
>>>> time formating	..................
>>>> time elapsed	hrs: 0 min: 02 sec: 03.00


In [16]:
#In this notebook we are going to train a simple multi-layer perceptrons on MNIST dataset:
#Model bulding and training loop are all created from scatch with pytorch functionalities:


In [17]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.functional as F
import time, random, sys
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from tqdm import tqdm


In [18]:
#The following class create our 3 layers MLP:
class MLP(nn.Module):
  def __init__(self, input_dim, num_class):
    super(MLP, self).__init__()
    self.fc1 = nn.Linear(in_features = input_dim, out_features = 128)
    self.fc2 = nn.Linear(in_features = 128, out_features = 64)
    self.relu = nn.ReLU()
    self.fc3 = nn.Linear(in_features = 64, out_features = 32)
    self.softmax = nn.Softmax()
    self.out = nn.Linear(in_features = 32, out_features = num_class)

  def forward(self, input_tensor):
    x = self.relu(self.fc1(input_tensor))
    x = self.relu(self.fc2(x))
    x = self.relu(self.fc3(x))
    x = self.softmax(self.out(x))
    return x

    

In [19]:
#Instantiate and testing the class :
input_dim = 784
num_class = 10
rand_data = torch.rand(size = (64, 784))
mlp = MLP(input_dim, num_class).to(device = device)
print(f">>> output shape: {mlp(rand_data).shape}")

>>> output shape: torch.Size([64, 10])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()


In [20]:
#Import the data from torchvision and preprocess it using transform method:
batch_size = 128
EPOCHS = 10
learning_rate = 1e-3
train_dfm = datasets.MNIST(root = 'mnist_mlp_train/', train = True, transform = transforms.ToTensor(), download = True)
test_dfm = datasets.MNIST(root = "mnist_mlp_test/",train = False, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_dfm, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dfm, batch_size = batch_size, shuffle = False)
x_train_batch, y_train_batch = next(iter(train_loader))
print(f">>>> x_train_shape: {x_train_batch.shape}\ty_train_shape: {y_train_batch.shape}")

>>>> x_train_shape: torch.Size([128, 1, 28, 28])	y_train_shape: torch.Size([128])


In [21]:
#Get the loss and optimizer's object:
loss = nn.CrossEntropyLoss()
optimizer = optim.Adam(params = mlp.parameters(), lr = learning_rate)

In [25]:
#The training loop 
tic = time.time()
for epoch in range(EPOCHS):
  print(f"\n>>>> training starts for epoch {epoch + 1}\n>>>> please wait while the model is training...............")
  for idx, (data, target) in enumerate(tqdm(train_loader)):
    data = data.to(device = device)
    target = target.to(device = device)
    data = data.reshape(data.shape[0],-1)
    #forward pass
    preds = mlp(data)
    train_loss = loss(preds, target)
    optimizer.zero_grad()
    #backward pass
    train_loss.backward()
    #gradient descent with adam optimizer
    optimizer.step()

def __valid__(loder, model):
  num_examples = 0
  num_correct = 0
  model.eval()
  #No need to compute the gradients again
  with torch.no_grad():
    for x, y in loder:
      x = x.to(device = device)
      y = y.to(device = device)
      x = x.reshape(x.shape[0], -1)
      preds = model(x) #get the predictions
      _, predictions = preds.max(1) #Get the maximum prob amoung the class of 10 probs members
      num_correct+=(predictions == y).sum() #Total numer of correctly predicted class
      num_examples+=predictions.size(0)
    acc = (num_correct/num_examples) * 100
    print(f">>>> at batch {idx + 1} of epoch {epoch + 1}\n>>>> loss: {train_loss:.4f}, accuracy: {acc:4f}")
  model.train()

__valid__(train_loader, mlp)
__valid__(test_loader, mlp)

    

  0%|          | 0/469 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
  1%|▏         | 7/469 [00:00<00:07, 65.34it/s]


>>>> training starts for epoch 1
>>>> please wait while the model is training...............


  1%|▏         | 6/469 [00:00<00:08, 57.02it/s]


>>>> training starts for epoch 2
>>>> please wait while the model is training...............


  1%|▏         | 7/469 [00:00<00:07, 63.59it/s]

...


>>>> training starts for epoch 9
>>>> please wait while the model is training...............


  1%|▏         | 7/469 [00:00<00:06, 66.22it/s]


>>>> training starts for epoch 10
>>>> please wait while the model is training...............


100%|██████████| 469/469 [00:06<00:00, 67.77it/s]


>>>> at batch 469 of epoch 10
>>>> loss: 1.4902, accuracy: 99.005005
>>>> at batch 469 of epoch 10
>>>> loss: 1.4902, accuracy: 97.509995
